In [1]:
import anndata
import os
import requests

save_path = "data/atac.h5ad"
if not os.path.exists(save_path):
    response = requests.get("https://go.wisc.edu/7gjjag")
    with open(save_path, "wb") as f:
        f.write(response.content)

exper = anndata.read_h5ad(save_path)
exper = exper[:1000, :]

In [2]:
from scdesigner.simulators import NegBinCopulaSimulator

sim = NegBinCopulaSimulator(epochs=10)
sim.fit(exper, "~ cell_type", copula_groups="cell_type")
samples = sim.sample(exper.obs)

In [3]:
import numpy as np

def tf_idf(y):
    y_sum = np.sum(y, axis=0)
    nfreqs = np.divide(y, y_sum, where=y_sum != 0)
    nfreqs[np.isnan(nfreqs)] = 0

    idf = 1 + np.log(1 + y.shape[1]) - np.log(1 + np.sum(y > 0, axis=1))
    return nfreqs * idf[:, np.newaxis]

In [4]:
import altair
from scdesigner.diagnose import compare_umap
altair.data_transformers.enable("vegafusion")

compare_umap(exper, samples, transform=tf_idf)

alt.FacetChart(...)